In [1]:
# TODO: Ta fram data från några utvalda kryptovalutor - Använd binance API (se my_crypto.py)
# TODO: Gör viss preprocessing per kryptovaluta till separate df
# TODO: Gör en merge av alla kryptovalutors df
# TODO: Sätt tomorrows up/down (1/0) till en kolumn i df som target
# TODO: Lägg till inflationskolumnen

# TODO: Gör en split av df till train och test
# TODO: Gör en GridSearchCV för en pipeline som innehåller en StandardScaler för olika modeller
# TODO: Skall vi gå vidare med övermorgon också?

# TODO: Gå vidare med logiken i my_crypto.py

In [2]:
import sklearn

import xgboost as xgb

import catboost
from catboost import CatBoostClassifier

# from pytrends.request import TrendReq
# import pytrends
import sys
import numpy as np
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
import ta
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score    
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
# plt.style.use('fivethirtyeight')
pd.set_option('display.width', 200)
import plotly.express as px
import streamlit as st
# import google

from IPython.display import display
# set display witdh to 200
pd.set_option('display.width', 200)
# set display height to 500
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = 200    # default 20
pd.options.display.max_colwidth = 60  # default 50
# pd.options.display.precision = 4      # default 6
import logging

# Starta upp logging och inkluder rad nummer
logging.basicConfig(level=logging.INFO, filemode='w', filename='krypto.log', force=True,
                    encoding='utf-8', format= '%(asctime)s - %(levelname)s - %(lineno)d - %(message)s')
logging.info('Startar')


In [3]:
print('Versioner av softvara som används i denna notebook:')
# print numpy version
print('numpy=={}'.format(np.__version__))
# print pandas version
print('pandas=={}'.format(pd.__version__))
# print matplotlib version
print('matplotlib=={}'.format(plt.matplotlib.__version__)) # type: ignore
# print yfinance version
print('yfinance=={}'.format(yf.__version__))
# print pytreands version
print('pytrends==4.8.0') 
# print python-dateutil version
print('python-dateutil==2.8.2')
# print ta version
print('ta==0.10.2')
# print streamlit version
print('streamlit=={}'.format(st.__version__))
print('xgboost=={}'.format(xgb.__version__))
print('catboost=={}'.format(catboost.__version__))

print('sklearn=={}'.format(sklearn.__version__))
#print google version
print('google-crc32==1.1.2')
# print pandas-ta version
print('pandas-ta==0.3.14b0')
# print plotly version
print('plotly==5.3.1')



Versioner av softvara som används i denna notebook:
numpy==1.23.5
pandas==1.4.4
matplotlib==3.7.0
yfinance==0.2.12
pytrends==4.8.0
python-dateutil==2.8.2
ta==0.10.2
streamlit==1.14.0
xgboost==1.7.1
catboost==1.1.1
sklearn==1.0.2
google-crc32==1.1.2
pandas-ta==0.3.14b0
plotly==5.3.1


In [ ]:
# TODO Vilka kryptovalutor ska vi använda?
tickers = ['BTC-USD','ETH-USD','BCH-USD','XRP-USD','ZRX-USD']
horizons=[2,5,15,30,60,90,250]

In [ ]:
def beräkna_rolling_mean(df, ticker_names, window=20):
    # Funktionen används inte just nu
    rolling_mean=pd.DataFrame()
    for ticker_name in ticker_names:
        df[ticker_name+'-rolling']= df[ticker_name].rolling(window=window).mean()
        
    df.dropna(inplace=True)   
    return df   

def beräkna_cumulative_return(df, ticker_names):
    # Funktionen används inte just nu
    cumulative_return={}
    for ticker_name in ticker_names:
        cumulative_return[ticker_name] = (df[ticker_name].iloc[-1]/df[ticker_name].iloc[0]-1)
    return cumulative_return

## Get inflation data US and SE

In [ ]:
def add_horizon_columns(inflation, horizons):
    # print(horizons)
    for horizon in  horizons:
        # print(horizon)
        inflation['US_inflation_'+str(horizon)] = inflation['US_inflation'].rolling(horizon, 1).mean()
        inflation['SE_inflation_'+str(horizon)] = inflation['SE_inflation'].rolling(horizon, 1).mean()
                
        # print(inflation.columns)        
    return inflation


def initiate_data(inflation, df_dates, lang_dict, value_name):
    # display(inflation)
    inflation = inflation.melt(id_vars=['Year'], var_name='month', value_name=value_name)

    # use lang_dict to translate month names to numbers
    inflation['month'] = inflation['month'].map(lang_dict)

    inflation['date'] = pd.to_datetime(inflation['Year'].astype(str) + '-' + inflation['month'].astype(str))
    inflation.set_index('date', inplace=True)
    inflation.drop(['Year', 'month'], axis=1, inplace=True)
    inflation = df_dates.merge(inflation, how='left', left_on='date', right_index=True)
    inflation.set_index('date', inplace=True)
    inflation[value_name] = inflation[value_name].astype(str)
    inflation[value_name] = inflation[value_name].str.replace(',', '.')
    inflation[value_name] = inflation[value_name].str.replace(chr(8209), chr(45))
    inflation[value_name] = inflation[value_name].astype(float)
    inflation[value_name].interpolate(method='linear', inplace=True)
    return inflation


@st.cache_data
def get_inflation_data(graph=False):
    # Explain this function here        
        

    df_dates = pd.DataFrame(pd.date_range('1988-12-01', pd.to_datetime('today').date()), columns=['date'])
    
    US_inflation = pd.read_html('https://www.usinflationcalculator.com/inflation/current-inflation-rates/')
    US_inflation = US_inflation[0]
    # replace the cell including string starting with "Avail" with the NaN
    US_inflation.replace(to_replace=r'^Avail.*$', value=np.nan, regex=True, inplace=True)
    # set the first row as the header and drop the first row
    US_inflation.columns = US_inflation.iloc[0]
    US_inflation.drop(US_inflation.index[0], inplace=True)
    US_inflation.drop('Ave', axis=1, inplace=True)

    SE_inflation = pd.read_html('https://www.scb.se/hitta-statistik/statistik-efter-amne/priser-och-konsumtion/konsumentprisindex/konsumentprisindex-kpi/pong/tabell-och-diagram/konsumentprisindex-med-fast-ranta-kpif-och-kpif-xe/kpif-12-manadersforandring/')
    SE_inflation = SE_inflation[0]
    SE_inflation.rename(columns={'År': 'Year'}, inplace=True)

    se_dict = dict(Jan='1', Feb='2', Mar='3', Apr='4', Maj='5', Jun='6', Jul='7', Aug='8', Sep='9', Okt='10', Nov='11', Dec='12')
    us_dict = dict(Jan='1', Feb='2', Mar='3', Apr='4', May='5', Jun='6', Jul='7', Aug='8', Sep='9', Oct='10', Nov='11', Dec='12')
    #
    SE_inflation = initiate_data(SE_inflation, df_dates, se_dict, value_name='SE_inflation')
    SE_inflation['SE_inflation'] = SE_inflation['SE_inflation'] / 10  # SE_inflation is in percent, divide by 10 to get decimal
    US_inflation = initiate_data(US_inflation, df_dates, us_dict,  value_name='US_inflation')

    if graph:
        # Do a simple plot
        fig, ax = plt.subplots(figsize=(12, 10))
        ax.plot(SE_inflation.index, SE_inflation['SE_inflation'], label='Sverige')
        ax.plot(US_inflation.index, US_inflation['US_inflation'], label='USA')
        ax.set_xlabel('Date')
        ax.set_ylabel('Inflation')
        ax.legend()
        plt.show()

    # concat and set one column to US_index and the other to SE_index
    inflations = pd.concat([US_inflation, SE_inflation], axis=1)
    inflations = inflations.dropna()
    inflations = add_horizon_columns(inflations, [75,90,250])
    return inflations

inflations = get_inflation_data(graph=True)

In [ ]:
display(list(inflations.columns))

In [ ]:
# fill up a dataframe with all days from 2005 up to today and set it as index
def get_all_dates():
    start_date = dt(2005, 1, 1)
    end_date = dt.today()
    dates = pd.date_range(start_date, end_date)
    df = pd.DataFrame(index=dates)
    return df


## Ladda ner all data

In [ ]:
def get_all(tickers):
    # Funktionen används inte just nu    
    df = pd.DataFrame()
    for ticker in tickers:
        # all_tickers = yf.download(ticker)
        df[ticker] = yf.download(ticker, period='max')['Close']
        
    df.index = pd.to_datetime(df.index)
    return df

df_org = get_all(tickers)

In [ ]:

def fit_predict(train, test, target, predictors, model, proba_val=None):
    
    if train[target].nunique() < 2:
        print('kolumner i train',train.columns)
        print('kolumner i test',test.columns)
        print('target',target)
        print('predictors',predictors)
        print('train\n',train[target].value_counts())
        print('test\n',test[target].value_counts())
        print(
            f"Skipping training for target {target} as it has only one unique value in the training data.")
        # stäng ner programmet
        sys.exit()
        
    model.fit(train[predictors], train[target])

    if proba_val:  # använd proba
        preds = model.predict_proba(test[predictors])[:, 1]
        preds[preds >= proba_val] = 1
        preds[preds < proba_val] = 0
    else:    # använd predict utan proba
        preds = model.predict(test[predictors])

    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test[target], preds], axis=1)

    return combined


### Beräkna ny kolumner (predictors)

In [ ]:
def create_predictors(df_, ticker, target, horizons=[2,5,60,250], ETH_BTC=False):
    df = df_.copy()
    ticker_name = ticker[:3]
    # tidsintervall i dagar för rullande medelvärden
    # skulle helst ha med upp till 4 år men ETH har för få värden
    predictors=[]
     
    if ETH_BTC: 
        df['ETH_BTC_ratio'] = df['ETH-USD']/df['BTC-USD']
        predictors += ['ETH_BTC_ratio']
        
        df['ETH_BTC_lag1'] = df['ETH_BTC_ratio'].shift(1)
        predictors += ['ETH_BTC_lag1']
        
        df['ETH_BTC_lag2'] = df['ETH_BTC_ratio'].shift(2)
        predictors += ['ETH_BTC_lag2']
        
        if ticker not in ['BTC-USD', 'ETH-USD']:
            df[ticker_name+'_BTC_ratio'] = df[ticker]/df['BTC-USD']
            predictors += [ticker_name+'_BTC_ratio']
        
            df[ticker_name+'_BTC_lag1'] = df[ticker_name+'_BTC_ratio'].shift(1)
            predictors += [ticker_name+'_BTC_lag1']
            
            df[ticker_name+'_BTC_lag2'] = df[ticker_name+'_BTC_ratio'].shift(2)
            predictors += [ticker_name+'_BTC_lag2']
            
            df[ticker_name+'_ETH_ratio'] = df[ticker]/df['ETH-USD']
            predictors += [ticker_name+'_ETH_ratio']
        
            df[ticker_name+'_ETH_lag1'] = df[ticker_name+'_ETH_ratio'].shift(1)
            predictors += [ticker_name+'_ETH_lag1']
            
            df[ticker_name+'_ETH_lag2'] = df[ticker_name+'_ETH_ratio'].shift(2)
            predictors += [ticker_name+'_ETH_lag2']
    
    # Target
    # tomorrow's close price - alltså nästa dag
    df['Tomorrow'] = df[ticker].shift(-1)
    # after tomorrow's close price - alltså om två dagar
    df['After_tomorrow'] = df[ticker].shift(-2)
    df['y1'] = (df['Tomorrow'] > df[ticker]).astype(int)
    df['y2'] = (df['After_tomorrow'] > df[ticker]).astype(int)
    # df.dropna(inplace=True)

    for horizon in horizons:
        rolling_averages = df.rolling(horizon, min_periods=1).mean()

        ratio_column = f"Close_Ratio_{horizon}"
        df[ratio_column] = df[ticker] / rolling_averages[ticker]
        
        rolling = df.rolling(horizon,closed='left', min_periods=1).mean()
        
        trend_column = f"Trend_{horizon}"
        target_name = 'Tomorrow' if target=='y1' else 'After_tomorrow'
        df[trend_column] = rolling[target_name]  

        predictors += [ratio_column, trend_column]

    df['day_of_month'] = df.index.day
    predictors += ['day_of_month']
    
    df['day_of_week'] = df.index.dayofweek
    predictors += ['day_of_week']
    
    df['month'] = df.index.month
    predictors += ['month']

    return df, predictors


Grafer för uppåt och nedåt predictions

# Skapa en GridSearchCV för olika modeller

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


In [ ]:
### get the data ###
df_grid = get_all(tickers)
df_inflations = get_inflation_data(graph=False)

In [ ]:
### preprocessing ###
# Vi gör GridSearch för enbart en kryptovaluta i taget. Vi väljer BTC och ETH.
ticker = tickers[1]
print(ticker)
target='y1'  # tomorrow
df, predictors = create_predictors(df_grid, ticker, target, horizons=horizons, ETH_BTC=False)

# inflation data #
df = pd.concat([df, df_inflations], axis=1)
df.dropna(subset=['Tomorrow'], inplace=True)
df.dropna(subset=[target], inplace=True)

print(df[['Tomorrow']].values > df[[ticker]].values)
print(df.shape)


In [ ]:
predictors = ['Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5', 'Close_Ratio_15',
       'Trend_15', 'Close_Ratio_30', 'Trend_30', 'Close_Ratio_60', 'Trend_60', 'Close_Ratio_90', 'Trend_90', 'Close_Ratio_250', 'Trend_250', 'US_inflation',
       'SE_inflation', 'US_inflation_75', 'SE_inflation_75', 'US_inflation_90', 'SE_inflation_90', 'US_inflation_250', 'SE_inflation_250']

tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
y = df['y1']
X = df.drop('y1', axis=1)

# split_index is the row-index that splits the data into 75% for training and 25% for testing
split_index = int(0.9 * len(df))

# split the data into training and testing sets
X_train = X.iloc[:split_index, :]
y_train = y.iloc[:split_index]
X_test = X.iloc[split_index:, :]
y_test = y.iloc[split_index:]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
params_xgb = {
    'max_depth': [ 3, 4, 5],
    'n_estimators': [ 50, 100, 150],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.15]
}

params_cat = {
    'depth': [7, 10, 12 ],
    'iterations': [ 30, 50, 100],
    'learning_rate': [0.01, 0.03, 0.05, 0.1]
}


In [ ]:
### GridSearchCV  ###
if True:
    xgb = XGBClassifier()
    grid_xgb = GridSearchCV(estimator=xgb, param_grid=params_xgb, n_jobs=-1, pre_dispatch='2*n_jobs', cv=tscv, 
                            scoring=['roc_auc', 'neg_log_loss', 'precision', 'f1'], refit='roc_auc', verbose=0)
    grid_xgb.fit(X_train[predictors], y_train)
print('xgb',grid_xgb.best_score_, grid_xgb.best_params_)

if True:
    cat = CatBoostClassifier()
    grid_cat = GridSearchCV(estimator=cat, param_grid=params_cat, n_jobs=-1,pre_dispatch='2*n_jobs', cv=tscv, # type: ignore
                             scoring=['roc_auc', 'neg_log_loss', 'precision', 'f1'], refit='roc_auc',verbose=0) 
    grid_cat.fit(X_train[predictors], y_train) 
print('cat',grid_cat.best_score_, grid_cat.best_params_)

In [ ]:
# sorted(sklearn.metrics.SCORERS.keys())
print(grid_xgb.best_score_, grid_xgb.best_index_ , grid_xgb.best_params_)
print(grid_cat.best_score_, grid_cat.best_index_ , grid_cat.best_params_)

In [ ]:
import sklearn.metrics

def log_loss(y_true, y_pred):
    return sklearn.metrics.log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None) 

y_pred_xgb = grid_xgb.predict(X_test[predictors])
y_pred_cat = grid_cat.predict(X_test[predictors])

dict1 = {"XGBoost": {'f1':f1_score(y_test, y_pred_xgb), 'AUC': roc_auc_score(y_test, y_pred_xgb), 
                     'precision': precision_score(y_test, y_pred_xgb), 'recall': recall_score(y_test, y_pred_xgb),
                     'log_loss': log_loss(y_test, y_pred_xgb)},
         "catboost":{'f1':f1_score(y_test, y_pred_cat), 'AUC': roc_auc_score(y_test, y_pred_cat), 
         'precision': precision_score(y_test, y_pred_cat), 'recall': recall_score(y_test, y_pred_cat),
         'log_loss': log_loss(y_test, y_pred_cat)}}
print(pd.DataFrame(dict1))


In [ ]:
pd.DataFrame(dict1).to_csv(ticker[:3]+'_output.csv',index=True)

BTC:  
- XGB: 0.5198207468963943 9 {'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 50}  
- CAT: 0.5362153381013091 15 {'depth': 10, 'iterations': 30, 'learning_rate': 0.1}  
  
ETH:  
- XGB: 0.5427315401213113 4 {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}  
- CAT: 0.5477744412767092 25 {'depth': 12, 'iterations': 30, 'learning_rate': 0.03}  
     
Slutsats:     
__CatBoost__ är möjligen bättre än XGBoost för de flesta score för BTC men inte lika tydligt för ETH